In [1]:
# 1. Python 标准库模块
import os
import sys
import time

# 2. Python 第三方模块
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 3. 应用程序自定义模块
from visualization import TrainingVisualizer

## 加载数据集

In [2]:
# 数据预处理和加载
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
'''
这段代码定义了一个 transform 对象，用于对数据进行预处理。这里使用了两个预处理步骤：

transforms.ToTensor():

将PIL Image或numpy ndarray转换为tensor（张量），并且将像素值从0-255缩放到0-1之间。
这个操作对于深度学习模型是必要的，因为模型期望输入是浮点型的张量。
transforms.Normalize((0.5,), (0.5,)):

对张量进行标准化处理。
第一个参数 (0.5,) 是均值，第二个参数 (0.5,) 是标准差。它们都是针对每个通道的。
对于Fashion MNIST数据集（灰度图像，只有一个通道），每个像素点会按公式 (x - mean) / std 进行标准化，这里将其变换为 (x - 0.5) / 0.5，即将像素值从范围 [0, 1] 变换到范围 [-1, 1]。
'''

train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

In [3]:
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
len(test_loader) # 总元素个数要乘上batch_size

40

## 定义模型

In [5]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
# 初始化模型、损失函数和优化器
model = LeNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [7]:
# 训练和测试模型
def train(model, device, train_loader, optimizer, criterion, epoch, visualizer):
    model.train()
    train_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
            
    avg_train_loss = train_loss / len(train_loader)
    train_accuracy = correct / len(train_loader.dataset)
    
    # visualizer.add(epoch, (avg_train_loss, train_accuracy))
    return epoch, avg_train_loss, train_accuracy

def test(model, device, test_loader, criterion, epoch, visualizer):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}'
          f' ({test_accuracy:.0f}%)\n')
    
    # visualizer.add(epoch, (None, None, test_accuracy))
    return epoch, test_accuracy

In [8]:
# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
# 初始化 TrainingVisualizer
visualizer = TrainingVisualizer(xlabel='Epoch', ylabel='Value', title='Training and Test Metrics',
                                legend=['Train Loss', 'Train Accuracy', 'Test Accuracy'])

# 训练和测试模型
for epoch in range(1, 101):
    epoch, avg_train_loss, train_accuracy = train(model, device, train_loader, optimizer, criterion, epoch, visualizer)
    epoch, test_accuracy = test(model, device, test_loader, criterion, epoch, visualizer)
    visualizer.add(epoch, (avg_train_loss, train_accuracy, test_accuracy))

Train Epoch: 71 [0/60000 (0%)]	Loss: 0.125804
Train Epoch: 71 [25600/60000 (43%)]	Loss: 0.064712
